In [1]:
from textblob import TextBlob
from Extract import Extract
import json
import spacy

In [2]:
extract = Extract('Apple')

In [3]:
news_extract = extract.get_news_extract().json()


In [4]:
spacy_nlp = spacy.load('en_core_web_sm')


In [5]:
def get_ner_dict(description):
        ner_list = []
        spacy_nlp = spacy.load('en')
        document = spacy_nlp(description)
        for element in document.ents:
            ner_list.append(str(element))
        return ner_list

In [6]:
for article in news_extract['articles']:
    ner_tags = get_ner_dict(article['description'])
    article['ner_tags'] = ner_tags


In [7]:
news_extract['articles']

[{'source': {'id': 'cnbc', 'name': 'CNBC'},
  'author': 'Reuters',
  'title': "US to probe proposed French tech tax, concerned it 'unfairly targets American companies'",
  'description': '"The United States is very concerned that the digital services tax which is expected to pass the French Senate tomorrow unfairly targets American companies," U.S. Trade Representative Robert Lighthizer said.',
  'url': 'https://www.cnbc.com/2019/07/11/us-to-probe-proposed-french-tech-tax.html',
  'urlToImage': 'https://image.cnbcfm.com/api/v1/image/103168008-IMG_0558-1.jpg?v=1531831865',
  'publishedAt': '2019-07-11T00:18:00Z',
  'content': 'U.S. President Donald Trump on Wednesday ordered an investigation into France\'s planned tax on technology companies, a probe that could lead to the United States imposing new tariffs or other trade restrictions.\r\n "The United States is very concerned that the… [+2499 chars]',
  'ner_tags': ['The United States',
   'French',
   'Senate',
   'tomorrow',
   'Ameri

In [8]:
from elasticsearch import Elasticsearch

In [9]:
 es = Elasticsearch([{'host': 'localhost', 'port': '9200'}])

In [10]:
for article in news_extract['articles']:
    es.index(index='news_data', ignore=400, doc_type='external', body={
        'timestamp': article['publishedAt'],
        'title': article['title'],
        'description': article['description'],
        'url': article['url'],
        'ner_tags': article['ner_tags'],
        'company_name': 'Apple',

    })

In [11]:
result = es.search(index='news_data', body={"query": {"match": {"ner_tags": "Apple"}}})
print(result)

{'took': 818, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 17, 'relation': 'eq'}, 'max_score': 1.5197494, 'hits': [{'_index': 'news_data', '_type': 'external', '_id': 'UKJm3msBfZVhATRrqDwt', '_score': 1.5197494, '_source': {'timestamp': '2019-07-10T22:47:00Z', 'title': 'Apple quietly updates Macs to remove Zoom webcam exploit', 'description': "It's not just Zoom scrambling to fix the exploit that allowed intruders to hijack Mac webcams. Apple has issued a silent, automatic update to macOS that removes the web server used to streamline access to the video conferencing app. The update isn'...", 'url': 'https://www.engadget.com/2019/07/10/apple-mac-update-removes-zoom-exploit/', 'ner_tags': ['Zoom', 'Mac', 'Apple'], 'company_name': 'Apple'}}, {'_index': 'news_data', '_type': 'external', '_id': 'WaJm3msBfZVhATRrqTww', '_score': 1.5197494, '_source': {'timestamp': '2019-07-10T20:53:00Z', 'title': "How to listen to podca

In [1]:
import numpy as np